In [1]:
import os

# Set the CUDA_VISIBLE_DEVICES environment variable to specify which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from musiclm_pytorch import MuLaNTrainer, MuLaN

import pickle
import torch

from torch.utils.data import Dataset

import torch
from musiclm_pytorch import MuLaN, AudioSpectrogramTransformer, TextTransformer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-26 12:04:16.358537: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-26 12:04:16.360276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-26 12:04:16.397169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler 

In [3]:
audio_transformer = AudioSpectrogramTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
    spec_n_fft = 256,
    spec_win_length = 24,
    spec_aug_stretch_factor = 0.8
)

text_transformer = TextTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64
)

mulan = MuLaN(
    audio_transformer = audio_transformer,
    text_transformer = text_transformer
)

In [4]:

from torch.utils.data import Dataset
from pathlib import Path

class MuLanDataset(Dataset):
    def __init__(self, txt_pickle_path: Path, wav_pickle_path: Path):
                
        with open(wav_pickle_path, 'rb') as f:
            self.wavs = pickle.load(f)
        
        with open(txt_pickle_path, 'rb') as f:
            self.txts = pickle.load(f)

        self.num_data = len(self.txts)
                
    def __len__(self):
        return self.num_data
    
    def __getitem__(self, idx):
        # read wav from pt file, read txt from list
        return self.wavs[idx], self.txts[idx]

class MusicDataset(Dataset):
    def __init__(self, musiccap_txt_pickle_path: Path, musiccap_wav_pickle_path: Path, sdd_dataset_pkl_path: Path):
                
        with open(musiccap_wav_pickle_path, 'rb') as f:
            self.musiccap_wavs = pickle.load(f)
        
        with open(musiccap_txt_pickle_path, 'rb') as f:
            self.musiccaps_txts = pickle.load(f)

        self.num_musiccaps = len(self.musiccap_wavs)

        sdd_dataset = pickle.load(open(sdd_dataset_pkl_path, 'rb'))
        self.sdd_wavs = [x[0] for x in sdd_dataset]
        self.sdd_txts = [x[1] for x in sdd_dataset]
        self.sdd_wavlens = [x[2] for x in sdd_dataset]
        
        assert len(self.sdd_wavs) == len(self.sdd_txts) and len(self.sdd_wavlens) == len(self.sdd_txts)
        
        self.num_sdd = len(self.sdd_wavs)
        self.wav_duration = 16000 * 10 # 10 seconds
        
        self.num_data = len(self.musiccaps_txts) + len(self.sdd_txts)
        assert self.num_data == (self.num_musiccaps + self.num_sdd)
        
    def __len__(self):
        return self.num_data
    
    def __getitem__(self, idx):
        if idx < len(self.musiccaps_txts):
            return self.musiccap_wavs[idx], self.musiccaps_txts[idx]
        
        else:
            # get the index
            idx = idx - len(self.musiccaps_txts)
            real_wav_len = self.sdd_wavlens[idx]
            # randomly select a starting point
            start_point = torch.randint(0, real_wav_len - self.wav_duration, (1,)).item()
            wav = self.sdd_wavs[idx][start_point:start_point+self.wav_duration]
            cap = self.sdd_txts[idx]
            return wav, cap
    
    
# training_data = MuLanDataset(
#     txt_pickle_path=Path('pkls/txts.pkl'),
#     wav_pickle_path=Path('pkls/wavs.pkl'))

training_data = MusicDataset(
    musiccap_txt_pickle_path=Path('pkls/txts.pkl'),
    musiccap_wav_pickle_path=Path('pkls/wavs.pkl'),
    sdd_dataset_pkl_path=Path('pkls/sdd_dataset.pkl')
)

In [5]:
# test MusicDataset
num_musiccap_data = training_data.num_musiccaps
print(num_musiccap_data)
wav, txt = training_data[num_musiccap_data + 10]
print(wav.shape, txt)

wav, txt = training_data[10]
print(wav.shape, txt)

3687
torch.Size([160000]) This is an alternative rock song with slow tempo and guitar  male vocals and drums 
torch.Size([160000]) This is an RBsoul music piece There is a male vocalist singing melodically and in a sensual manner over multiple tracks The melody of the beat is provided by a string sample There is a strong bass sound in the piece The rhythm is provided by a groovy electronic drum beat There is an urban feel to this piece It could be used in the soundtrack of a TV series where two characters are sharing intimate moments


### Train Mulan

In [6]:
mulan_trainer = MuLaNTrainer(mulan=mulan, dataset=training_data, num_train_steps=1000, batch_size=16, grad_accum_every=16)

training with dataset of 4173 samples and validating with randomly splitted 220 samples


In [7]:
mulan_trainer.train()

spectrogram yielded shape of (129, 1251), but had to be cropped to (128, 1248) to be patchified for transformer
0: loss: 2.7250650376081467
0: saving model to results
0: saving model with minimum loss to results
1: loss: 3.680168643593788
2: loss: 2.72372467815876
2: saving model with minimum loss to results
3: loss: 2.7109251022338867
3: saving model with minimum loss to results
4: loss: 2.7091198414564133
4: saving model with minimum loss to results
5: loss: 2.707549527287483
5: saving model with minimum loss to results
6: loss: 2.7066099643707275
6: saving model with minimum loss to results
7: loss: 2.7059807032346725
7: saving model with minimum loss to results
8: loss: 2.7045385241508484
8: saving model with minimum loss to results
9: loss: 2.705275133252144
10: loss: 2.7030062675476074
10: saving model with minimum loss to results
11: loss: 2.7012705951929092
11: saving model with minimum loss to results
12: loss: 2.699175074696541
12: saving model with minimum loss to results
13

### Load saved mulan model

In [ ]:
mulan_trainer = MuLaNTrainer(mulan=mulan, dataset=training_data, num_train_steps=1000, batch_size=2, grad_accum_every=16)
min_loss_path = '/root/musiclm-pytorch/results/mulan_min_loss.pt'
mulan_trainer.load(min_loss_path)

### Get most sim music for the given text description

In [ ]:
from tqdm import tqdm

special_characters = {'&', ',', '"', "'", '/', ';', '“', '(', '‘', '’', '.', ')', '-', '\n', ':'}
def replace_special_characters_with_space(text):
    for char in special_characters:
        text = text.replace(char, ' ')
    return text
# input
query_text = ['This music features a classic piano solo, showcasing intricate melodies and expressive harmonies. The timeless elegance and nuanced performance create an immersive and captivating listening experience.']

query_text = [replace_special_characters_with_space(text) for text in query_text]


# get the latent representation of the query text
query_text_latent = mulan_trainer.mulan.get_text_latents(raw_texts=query_text)

# get the audio representation of the query text, highest similarity, iterate over all mulan dataset
max_similarity = 0
max_similarity_idx = 0
idx_simliarity_text_list = []
for idx in tqdm(range(len(training_data))):
    wav, txt = training_data[idx]
    # append fake batch
    wav = torch.unsqueeze(wav, 0).to(mulan_trainer.device)
    audio_latent = mulan_trainer.mulan.get_audio_latents(wav)
    
    # compute cosine similarity between two latents
    similarity = torch.nn.functional.cosine_similarity(query_text_latent, audio_latent).detach().cpu().numpy()
    
    idx_simliarity_text_list.append((idx, similarity, txt))
    
    if similarity > max_similarity:
        max_similarity = similarity
        max_similarity_idx = idx
        


In [ ]:
import IPython.display as ipd
from IPython.display import display

# sort the similarity_text list by similarity (higest to lowest)
print(query_text)
idx_simliarity_text_list.sort(key=lambda x: x[1], reverse=True)
for sim_text in idx_simliarity_text_list[:10]:
    idx, sim, txt = sim_text
    print(f'{idx} {sim} - {txt}')
    # display(ipd.Audio(training_data[idx][0], rate=16000))

In [ ]:
# show the best match
print(training_data[max_similarity_idx][1])
# play the audio of the best match
import IPython.display as ipd
ipd.Audio(training_data[max_similarity_idx][0], rate=16000)

### Text Encoder

In [ ]:

'''
text1 = ['his voice.  song']
text2 = ['his voice   \n  song']
text3 = ['his voice song']

embed1 = mulan.get_text_latents(raw_texts=text1)
print(embed1.shape)
embed2 = mulan.get_text_latents(raw_texts=text2)
print(embed2.shape)
embed3 = mulan.get_text_latents(raw_texts=text3)

print(embed1.sum())
print(embed2.sum())
print(embed3.sum())
'''
